In [46]:
import sqlite3
import os
import openpyxl as px
import numpy as np

In [65]:
# CONSTANTS
TOP = "G7"
LM = "G8"
MM = "H8"
RM = "I8"
LO = "G9"
MO = "H9"
RO = "I9"
LLO = "G10"
LRO = "H10"
MLO = "I10"
MRO = "J10"
RLO = "K10"
RRO = "L10"


jar_file = "\"/Users/dennismaes/Library/Mobile Documents/com~apple~CloudDocs/School/2021-2022/Thesis/applicatie/thesis/target/scheduler-1.0.0.jar:/Users/dennismaes/.m2/repository/org/xerial/sqlite-jdbc/3.34.0/sqlite-jdbc-3.34.0.jar:/Users/dennismaes/.m2/repository/org/javatuples/javatuples/1.2/javatuples-1.2.jar\""

In [2]:
db_path = "\"/Users/dennismaes/Library/Mobile\ Documents/com~apple~CloudDocs/School/2021-2022/Thesis/applicatie/scheduler/backend/demo.db\""
db_path

'"/Users/dennismaes/Library/Mobile\\ Documents/com~apple~CloudDocs/School/2021-2022/Thesis/applicatie/scheduler/backend/demo.db"'

In [3]:
rpath = "../scheduler/backend/demo.db"

In [28]:
conn = sqlite3.connect(rpath)

In [29]:
cur = conn.cursor()

In [90]:
RESULTS_FILE= "results_tests.xlsx"


wknd_shifts = ["JAWE", "SAWE", "TPWE"]
week_shifts = ["CALL", "JANW", "SAEW", "JAEV"]

WL = dict()
for sh in wknd_shifts:
    WL[sh] = 1.5
for sh in week_shifts:
    WL[sh] = 1

    
def parse_excel_input(ws):
    return ws[TOP].value, ws[LM].value, ws[MM].value, ws[RM].value, ws[LO].value, ws[MO].value, ws[RO].value, ws[LLO].value, ws[LRO].value, ws[MLO].value, ws[MRO].value, ws[RLO].value, ws[RRO].value
    
    
def write_to_excel(results_dict):
    wb = px.load_workbook(RESULTS_FILE)
    print(wb.worksheets)
    wb.create_sheet(index=len(wb.sheetnames), title="results "+  str(len(wb.worksheets)))
    ws = wb["results "+  str(len(wb.worksheets) -1)]
    ws["C2"] = "Times < 12 rest days"
    ws["D2"] = "Total WL"
    row = 3
    for x in results_dict.keys():
        ws["B" + str(row)] = x
        ws["C" + str(row)] = results_dict[x][0]
        ws["D" + str(row)] = results_dict[x][1]
        row += 1
    
    ws["B" + str(row + 1)] = "TOTAL"
    ws["C" + str(row + 1)] = get_sum(results)
    ws["D" + str(row + 1)] = get_fair(results)
    
    ws["B" + str(row + 2)] = "STD DEV"
    ws["D" + str(row + 2)] = get_std(results)
    
    wb.save(RESULTS_FILE)
    
def write_to_excel_ws(results_dict, wb, ws):
    ws["C2"] = "Times < 12 rest days"
    ws["D2"] = "Total WL"
    row = 3
    for x in results_dict.keys():
        ws["B" + str(row)] = x
        ws["C" + str(row)] = results_dict[x][0]
        ws["D" + str(row)] = results_dict[x][1]
        row += 1
    
    ws["B" + str(row + 1)] = "TOTAL"
    ws["C" + str(row + 1)] = get_sum(results)
    ws["D" + str(row + 1)] = get_fair(results)
    
    ws["B" + str(row + 2)] = "STD DEV"
    ws["D" + str(row + 2)] = get_std(results)
    
    wb.save(RESULTS_FILE)
    
def get_nb_times_rest_less_than(assignments, min_bal=12):
    first_shift = True
    counter = 0
    total = 0
    for ass in assignments:
        if ass == "FREE":
            if not first_shift:
                counter += 1
        else:
            if not first_shift:
                if counter < min_bal and counter > 0:
                    total += 1
            else: 
                first_shift = False
            
            counter = 0
    return total

def get_total_wl(assignments):
    wl = 0.0
    for sh in [x for x in str(assignments).replace("[","").replace("]", "").replace(",","").replace('FREE', "|").split("|") if x != "\'" and x != "\' \'"]:
        wl += WL[sh.split(" ")[1].replace("\'", "")]
    return wl

def get_std(results):
    return np.std([results[x][1] for x in results.keys()])


def get_sum(results):
    return np.sum([results[x][0] for x in results.keys() ])

def get_fair(results):
    return max([results[x][1] for x in results.keys()]) - min([results[x][1] for x in results.keys()])

In [105]:
def read_results(filename):
    with open(filename, 'r') as f:
        print(f.readlines()[-1])


def run_script():
    wb = px.load_workbook(RESULTS_FILE)
    for sh_n in wb.sheetnames:
        ws = wb[sh_n]
        input_data = parse_excel_input(ws)
        
        run_string = "java -cp " + jar_file + " Main"
        for x in input_data:
            run_string += " {}".format(x)
        
        #run_string += " > tmp.txt"
        
        os.system(run_string)
        
        schedule_entries = []
        for row in cur.execute('''SELECT assistant_id, day_nb, shift_type FROM assignment'''):
            schedule_entries.append(row)

        assistant_entries = dict()
        for row in cur.execute('''SELECT id, name FROM assistant'''):
            assistant_entries[row[0]] = row[1].strip()
        
        results = dict()
        for as_id in list(assistant_entries.keys()):
            current_assignments = [x for x in schedule_entries if x[0] == as_id]
            current_name = assistant_entries[as_id]
            shifts = [ x[2] for x in current_assignments]
            i = 0
            free_cnt = 0
            while i < len(shifts):
                if shifts[i] == 'FREE':
                    free_cnt += 1
                else:
                    if free_cnt == 2:
                        shifts[i-2] = 'JAEV'
                        shifts[i-1] = 'JAEV'
                    free_cnt = 0
                i += 1

            results[current_name] = (get_nb_times_rest_less_than(shifts), get_total_wl(shifts))
        write_to_excel_ws(results, wb, ws)
        #read_results("tmp.txt")

In [106]:
run_script()

In [114]:
print(os.getcwd())


/Users/dennismaes/Library/Mobile Documents/com~apple~CloudDocs/School/2021-2022/Thesis/applicatie/thesis


'do  3 mrt 2022 08:10:30 CET\n'

In [58]:
schedule_entries = []
for row in cur.execute('''SELECT assistant_id, day_nb, shift_type FROM assignment'''):
    schedule_entries.append(row)
    
assistant_entries = dict()
for row in cur.execute('''SELECT id, name FROM assistant'''):
    assistant_entries[row[0]] = row[1].strip()
assistant_entries

{24: 'Test JA 1',
 25: 'Test JAF 1',
 26: 'Test JA 2',
 27: 'Test JA 3',
 28: 'Test JA 4',
 29: 'Test JAF 2',
 30: 'Test JAF 3',
 31: 'Test SA 1',
 32: 'Test SA 2',
 33: 'Test SA 3',
 34: 'Test SAF 1',
 35: 'Test SAF 2',
 36: 'Test SAN 1',
 37: 'Test SAN 2',
 38: 'Test SAN 3',
 39: 'Test SAN 4',
 40: 'Test SAN 5',
 41: 'Test SAFN 1',
 42: 'Test SAFN 2',
 43: 'Test JA 5',
 44: 'Test JA 6',
 45: 'Test JA 7',
 46: 'Test JAF  4',
 47: 'Test SA 4',
 48: 'Test SA  5',
 49: 'Test SAF 3',
 50: 'Test SAN  6',
 51: 'Test SAN  7',
 52: 'Test SAFN 3',
 53: 'Test JA 8',
 54: 'Test JAF 5',
 55: 'Test SA 6',
 56: 'Test SAF 4',
 57: 'Test SAN 8',
 58: 'Test SAFN 4',
 59: 'Test JA 9',
 60: 'Test JA 10',
 61: 'Test SAFN 5',
 62: 'Test JAF 6',
 63: 'Test JA 11'}

In [59]:
results = dict()
for as_id in list(assistant_entries.keys()):
    current_assignments = [x for x in schedule_entries if x[0] == as_id]
    current_name = assistant_entries[as_id]
    shifts = [ x[2] for x in current_assignments]
    i = 0
    free_cnt = 0
    while i < len(shifts):
        if shifts[i] == 'FREE':
            free_cnt += 1
        else:
            if free_cnt == 2:
                shifts[i-2] = 'JAEV'
                shifts[i-1] = 'JAEV'
            free_cnt = 0
        i += 1
            
    results[current_name] = (get_nb_times_rest_less_than(shifts), get_total_wl(shifts))
    
results

{'Test JA 1': (1, 7.0),
 'Test JAF 1': (0, 7.5),
 'Test JA 2': (0, 7.5),
 'Test JA 3': (1, 7.5),
 'Test JA 4': (0, 8.0),
 'Test JAF 2': (0, 7.5),
 'Test JAF 3': (0, 7.5),
 'Test SA 1': (2, 8.0),
 'Test SA 2': (0, 8.0),
 'Test SA 3': (1, 8.0),
 'Test SAF 1': (0, 7.5),
 'Test SAF 2': (0, 7.5),
 'Test SAN 1': (1, 7.0),
 'Test SAN 2': (0, 7.5),
 'Test SAN 3': (1, 7.0),
 'Test SAN 4': (0, 7.0),
 'Test SAN 5': (1, 7.5),
 'Test SAFN 1': (0, 7.0),
 'Test SAFN 2': (0, 7.5),
 'Test JA 5': (0, 7.5),
 'Test JA 6': (0, 7.0),
 'Test JA 7': (0, 7.0),
 'Test JAF  4': (0, 7.5),
 'Test SA 4': (0, 7.5),
 'Test SA  5': (1, 7.0),
 'Test SAF 3': (0, 8.0),
 'Test SAN  6': (0, 7.0),
 'Test SAN  7': (0, 7.0),
 'Test SAFN 3': (0, 8.0),
 'Test JA 8': (1, 8.0),
 'Test JAF 5': (0, 7.5),
 'Test SA 6': (1, 7.0),
 'Test SAF 4': (0, 7.0),
 'Test SAN 8': (1, 7.5),
 'Test SAFN 4': (0, 7.5),
 'Test JA 9': (2, 7.5),
 'Test JA 10': (0, 8.0),
 'Test SAFN 5': (0, 7.5),
 'Test JAF 6': (0, 7.5),
 'Test JA 11': (0, 7.0)}

In [60]:
write_to_excel(results)

[<Worksheet "Blad1">, <Worksheet "results 1">, <Worksheet "results 2">, <Worksheet "results 3">]


AttributeError: Style objects are immutable and cannot be changed.Reassign the style with a copy

14